In [14]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from mezo.currency_utils import format_pool_token_columns, add_pool_usd_conversions, format_token_columns, add_usd_conversions
from mezo.datetime_utils import format_datetimes
from mezo.data_utils import add_rolling_values, flatten_json_column
from mezo.clients import BigQueryClient, SubgraphClient
from mezo.currency_config import POOLS_MAP, POOL_TOKEN_PAIRS, TOKENS_ID_MAP, TIGRIS_MAP
from mezo.visual_utils import ProgressIndicators, ExceptionHandler, with_progress
from mezo.queries import PoolQueries

In [15]:
POOLS_SUBGRAPH = "https://api.goldsky.com/api/public/project_cm6ks2x8um4aj01uj8nwg1f6r/subgraphs/musd-pools-mezo/1.1.0/gn"

SUBGRAPH_HEADERS = {
        "Content-Type": "application/json",
    }

In [3]:

# Get deposits data
deposits_data = SubgraphClient.get_subgraph_data(
    POOLS_SUBGRAPH, 
    PoolQueries.GET_DEPOSITS, 
    'mints'
)
# Get withdrawals data
withdrawals_data = SubgraphClient.get_subgraph_data(
    POOLS_SUBGRAPH, 
    PoolQueries.GET_WITHDRAWALS, 
    'burns'
)

# Get volume data
volume_data = SubgraphClient.get_subgraph_data(
    SubgraphClient.TIGRIS_POOLS_SUBGRAPH, 
    PoolQueries.GET_POOL_VOLUME, 
    'poolVolumes'
)
# Get fees data
fees_data = SubgraphClient.get_subgraph_data(
    SubgraphClient.TIGRIS_POOLS_SUBGRAPH, 
    PoolQueries.GET_TOTAL_POOL_FEES, 
    'feesStats_collection'
)

🔍 Trying mints query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
Fetching transactions with skip=2000...
Fetching transactions with skip=3000...
Fetching transactions with skip=4000...
Fetching transactions with skip=5000...
Fetching transactions with skip=6000...
Fetching transactions with skip=7000...
Fetching transactions with skip=8000...
No more records found.
✅ Found 7509 mints records
🔍 Trying burns query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
Fetching transactions with skip=2000...
Fetching transactions with skip=3000...
Fetching transactions with skip=4000...
No more records found.
✅ Found 3598 burns records
🔍 Trying poolVolumes query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records found.
✅ Found 276 poolVolumes records
🔍 Trying feesStats_collection query...
Fetching transactions with skip=0...
Fetching transactions with skip=1000...
No more records foun

In [ ]:
def process_pools_data(raw, transaction_type):
    """Process raw data from musd-pools subgraphs"""
    if not ExceptionHandler.validate_dataframe(raw, "Raw data", ['contractId_', 'timestamp_']):
        raise ValueError("Invalid data structure")
    
    df = raw.copy()
    df['pool'] = df['contractId_'].map(POOLS_MAP)
    df = format_datetimes(df, ['timestamp_'])
    df = format_pool_token_columns(df, 'contractId_', POOL_TOKEN_PAIRS)
    df = add_pool_usd_conversions(df, 'contractId_', POOL_TOKEN_PAIRS, TOKENS_ID_MAP)
    df['transaction_type'] = transaction_type

    return df

In [10]:
def process_volume_data(raw_volume):
    """Process raw volume data from Tigris pools"""
    if not ExceptionHandler.validate_dataframe(raw_volume, "Raw volume data", ['timestamp']):
        raise ValueError("Invalid volume data structure")
    
    df = raw_volume.copy()
    
    # Flatten JSON pool column
    df = flatten_json_column(df, 'pool')
    
    # Format timestamps
    df = format_datetimes(df, ['timestamp'])
    
    # Replace token symbols
    token_columns = ['pool_token0_symbol', 'pool_token1_symbol']

    for col in token_columns:
        df[col] = df[col].replace({
            'mUSDC': 'USDC',
            'mUSDT': 'USDT'
        })
    
    # Map pool names
    df['pool'] = df['pool_name'].map(TIGRIS_MAP)

    # Format token columns
    df = format_token_columns(df, ['totalVolume0'], 'pool_token0_symbol')
    df = format_token_columns(df, ['totalVolume1'], 'pool_token1_symbol')
    
    # Add USD conversions
    df = add_usd_conversions(df, 'pool_token0_symbol', TOKENS_ID_MAP, ['totalVolume0'])
    df = df.drop(columns=['index', 'usd'])
    df = add_usd_conversions(df, 'pool_token1_symbol', TOKENS_ID_MAP, ['totalVolume1'])
    df = df.drop(columns=['index', 'usd', 'pool_token0_symbol', 'pool_token1_symbol'])

    # Trim dataframe
    df = df[[
        'timestamp', 'pool_name', 'pool', 
        'totalVolume0', 'totalVolume1', 
        'totalVolume0_usd', 'totalVolume1_usd', 'id'
    ]]

    return df


In [5]:
deposits_clean = process_pools_data(deposits_data, 'deposit')
withdrawals_clean = process_pools_data(withdrawals_data, 'withdrawal')

✅ Raw data validation passed (7509 rows)
✅ Raw data validation passed (3598 rows)


In [11]:
volume_clean = process_volume_data(volume_data)

✅ Raw volume data validation passed (276 rows)


In [17]:
volume_clean

,timestamp,pool_name,pool,totalVolume0,totalVolume1,totalVolume0_usd,totalVolume1_usd,id
0,2025-09-25,Volatile AMM - BTC/MUSD,btc_musd_pool,0.156573,13905.103386,17154.470950,13905.103386,14795067118059521
1,2025-09-25,Stable AMM - mUSDC/MUSD,musdc_musd_pool,25625.924238,110549.180573,25618.364590,110549.180573,14794882434465793
2,2025-09-25,Stable AMM - MUSD/mUSDT,musd_musdt_pool,788.224030,112.961827,788.224030,112.961827,14777122744696833
3,2025-09-25,Stable AMM - mUSDC/mUSDT,musdc_musdt_pool,651.685939,769.059306,651.493692,769.059306,14770590099439619
4,2025-09-24,Volatile AMM - BTC/MUSD,btc_musd_pool,0.257813,25655.717912,28246.522235,25655.717912,14693495436476417
...,...,...,...,...,...,...,...,...
271,2025-07-08,Volatile AMM - BTC/MUSD,btc_musd_pool,0.000250,0.000000,27.390500,0.000000,6679932570697729
272,2025-07-07,Volatile AMM - BTC/MUSD,btc_musd_pool,0.000000,18.910000,0.000000,18.910000,6572738776924161
273,2025-07-07,Stable AMM - mUSDC/MUSD,musdc_musd_pool,2.010400,0.000000,2.009807,0.000000,6572554093330433
274,2025-06-26,Stable AMM - MUSD/mUSDT,musd_musdt_pool,0.000000,1.000000,0.000000,1.000000,5430148627169281


In [20]:
volume_clean.columns

Index(['timestamp', 'pool_name', 'pool', 'totalVolume0', 'totalVolume1',
       'totalVolume0_usd', 'totalVolume1_usd', 'id'],
      dtype='object')

In [ ]:
for pool in volume_clean['pool']:
    volume_clean.loc[volume_clean[pool]]

In [21]:
# Create separate dataframes for each pool
pools = {}

# Get unique pools
unique_pools = volume_clean['pool'].unique()
print(f"Found {len(unique_pools)} unique pools: {list(unique_pools)}")

# Create a separate dataframe for each pool
for pool_name in unique_pools:
    if pd.notna(pool_name):  # Skip NaN values if any
        pool_df = volume_clean[volume_clean['pool'] == pool_name].copy()
        pools[pool_name] = pool_df
        print(f"{pool_name}: {len(pool_df)} rows")

# Access individual pool dataframes like this:
pools['btc_musd_pool']
pools['musdc_musd_pool']

Found 5 unique pools: ['btc_musd_pool', 'musdc_musd_pool', 'musd_musdt_pool', 'musdc_musdt_pool', 'musdc_btc_pool']
btc_musd_pool: 81 rows
musdc_musd_pool: 75 rows
musd_musdt_pool: 73 rows
musdc_musdt_pool: 46 rows
musdc_btc_pool: 1 rows


,timestamp,pool_name,pool,totalVolume0,totalVolume1,totalVolume0_usd,totalVolume1_usd,id
1,2025-09-25,Stable AMM - mUSDC/MUSD,musdc_musd_pool,25625.924238,110549.180573,25618.364590,110549.180573,14794882434465793
5,2025-09-24,Stable AMM - mUSDC/MUSD,musdc_musd_pool,5796.119831,2315.415713,5794.409976,2315.415713,14693379472359425
8,2025-09-23,Stable AMM - mUSDC/MUSD,musdc_musd_pool,51780.437185,1826.671588,51765.161956,1826.671588,14582169884164097
12,2025-09-22,Stable AMM - mUSDC/MUSD,musdc_musd_pool,687.270488,10762.648355,687.067743,10762.648355,14465381133451265
15,2025-09-21,Stable AMM - mUSDC/MUSD,musdc_musd_pool,104.619600,5368.200000,104.588737,5368.200000,14376943461859329
...,...,...,...,...,...,...,...,...
257,2025-07-17,Stable AMM - mUSDC/MUSD,musdc_musd_pool,4969.702538,3796.893018,4968.236476,3796.893018,7603410668879873
258,2025-07-16,Stable AMM - mUSDC/MUSD,musdc_musd_pool,6221.213169,6036.966709,6219.377911,6036.966709,7507310775631873
265,2025-07-11,Stable AMM - mUSDC/MUSD,musdc_musd_pool,7.003515,2.003906,7.001449,2.003906,6969430546317313
270,2025-07-09,Stable AMM - mUSDC/MUSD,musdc_musd_pool,0.000000,2.009900,0.000000,2.009900,6724595935608833


In [ ]:

# total volume is the larger asset amount

# 24 h volume